In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
from warcio.capture_http import capture_http
import requests  # requests must be imported after capture_http
from warcio.archiveiterator import ArchiveIterator
from warcio import WARCWriter
import io

In [8]:
r = requests.get("https://www.eyedrum.org/calendar-events-performances-art-music/simone-baron-arco-belo-at-eyedrum")
f = open('test.html', 'w+')
f.write(r.text)
f.close()

In [4]:
with capture_http('test.warc.gz'):
    r = requests.get("https://fingerprinter-production.herokuapp.com/visitors/9da0eb5b2cac163039f1a7900f452aae/audience_segments")


### Replace target warc

In [7]:
warc = '/home/jingyz/Downloads/fidelity-issues.warc'
new_warc = '/home/jingyz/fidelity-issues.warc'
target_url_q = 'f4c9f.1f3ed359e8.js'
replace = lambda x: x.replace('cancelIdleCallback(w),', 'cancelIdleCallback(w), (()=>{throw Error("jingyz");})(),')

with open(warc, 'rb') as stream, open(new_warc, 'wb') as out:
    writer = WARCWriter(out, gzip=False)
    for record in ArchiveIterator(stream):
        uri = record.rec_headers.get_header('WARC-Target-URI')
        body = record.content_stream().read()
        if uri and target_url_q in uri and record.rec_type == 'response':
            print(body)
            body = replace(body.decode())
            body = body.encode()
            print(body)
        new_record = writer.create_warc_record(record.rec_headers.get_header('WARC-Target-URI'), 
                                            record.rec_type,
                                            payload=io.BytesIO(body),
                                            warc_headers=record.rec_headers,
                                            http_headers=record.http_headers)
        writer.write_record(new_record)